<a href="https://colab.research.google.com/github/Kalabras/2024/blob/main/v0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Object Detection


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/s?q=google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1%20OR%20google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub models</a>
  </td>
</table>

This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Setup


In [ ]:
#@title Imports and function definitions

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

## Example use

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [ ]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.LANCZOS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getbbox(ds)[3] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    bbox = font.getbbox(display_str)
    text_width, text_height = bbox[2], bbox[3]
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

## Apply module

Load a public image from Open Images v4, save locally, and display.

In [ ]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  #@param
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}
  print(result)

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)

In [ ]:
run_detector(detector, downloaded_image_path)

### More images
Perform inference on some additional images with time tracking.


In [ ]:
image_urls = [
  # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  "https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg",
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg",
  # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  "https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg",
  ]

def detect_img(image_url):
  start_time = time.time()
  image_path = download_and_resize_image(image_url, 640, 480)
  run_detector(detector, image_path)
  end_time = time.time()
  print("Inference time:",end_time-start_time)

In [ ]:
detect_img(image_urls[0])

In [ ]:
detect_img(image_urls[1])

In [ ]:
detect_img(image_urls[2])

In [ ]:
detect_img("data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFRYYGRgYGhkYGhoZHBgcGBwYGBgaGhoZHBkcIS4lHB4rHxgZJjgmKy8xNTU1GiU7QDs0Py40NTEBDAwMEA8QHhISHzQrIys0NDQ0NDQ0NDQ0MTQ0NDQ0NDQ0NDQ0NDQ0NDQ0NjY0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIAKIBNwMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAACAAEDBAUGB//EAD0QAAIBAgQEAwcDAQcEAwEAAAECEQADBBIhMQVBUWEicZEGEzKBobHwQsHRghRScrLC4fEjM2KSFaLDB//EABkBAQEBAQEBAAAAAAAAAAAAAAEAAgMEBf/EACYRAAICAgIBBAIDAQAAAAAAAAABAhEDEiExQQQTIlEyYXGR4UL/2gAMAwEAAhEDEQA/ANtVowKQFEBX0rPnJCAowKQFGBVZUMBRAUQFOBVY0DFOFogKICgWCFp8tEBRBarCgAtOBRxTxVZUBFPFFFPFVlQOWniiimiixSGilRKpPKaZ1jeix1BpqORTZhVZJDRTxRLH94DzzfsKlW0NPGmu2p/jSrZItWQRSiprtgruR9Y9YqNN6HLjgYw55Dt2tKspbHSkpEUz3xXFuUmelRjFEV0VBIFHceahK1uMX5OcpLwDcaajijIpRXVcI4SdsjIpxRRQxSZGmmJpyKYioRpoZoitCRUNjNQ05FARUQ7UqE01QBgUarTgUYFFmhgKICnAogKLGhgKICnC0QFVlQwFOBRAU8UWVDAUoogKICqwoGKeKKKeKrGgCKaidgN6ia50FRJBnvUZfpTEdaQPQVDQ49acrSBoWag0ICmYxTZvShXWmgsZaeaNbdIrTaKmJTSD0qLIeQNZshzdPKnS4DQ+7NMLBqVE02TxQkVEgYbenKpULc1+tVhqyNloYqwVoCtKkDi0RZaUVJFKKbCiEimIqUimIpsqISKUVIVoSKrKiMio2FTMKAiqyohIp6IilTZUTAUQWnAogK5WaoQFEFpxTgiqyoQFOBSkdRT5h1qsqHApwKH3g60xvdBSNEoFLbeoDdPWPKhIB3+utRUSvfA21oDdY9vvQ0oqKhstOBRKlELB/NqthoiqpxK/eVc9i2txgfEhbKzLzytsG20O+taP9mNGuGPWKHJV2aUWYo47aEC4t20TtnRgCQJIBG5rUS2HAZWBB1BG1R8Q4ct5Aj5sk5mCkqWjYGOX8CsHE324cVY5nwjNDGJe0XIEsQNVHXmJG8ZuW0om9Ys6dcP1FEmGHIVMlwQCDIIkEagg7EVOjipzkKiimbHem913q1cE7UJsmNTVsy0RAbZ5RTMGHKpGqNmpTCgINSIlAGpTSwok93Sio5pEd6KGw2aoXncGjnvQvtSuGT5RCMQeY9KkVgdjUJFDkFdeDjqWIpRUIcjn60YvdR6UBQRFCVos4608jrVZUQlaArVgigK1WVEJWlRkUqrKiuM3f1oxbJ/5FSUYNY2Z11RELRqW3ZHOjDVNbtg6k0OTNKKCtYQHy6zRHCgUSPGlOb1Y2ZrWJEbA6UwsCpi9NmrSkwcUAmFnYTUgwR6fWrdi8AKd8SKw5yHWJnvhDNRYiy8QkBj+ptQvfLIzHtp51bdydahZta1s2GqRy2N4JjLly3bOJL2Sc9xvd2lAyEMqRJLFjHKIB8q1jwK6pDpibucR/wBwh0KzqptgCJHQ6VspcApPf6VmjVjR3qPE4lUALTBZV0Vm1ZgokKDAk6nYUJesbjvFLtpMwtrlFy2M7PpDOoJyLB013I5b7USbjGyirdG1h8UriVBAll1VlMqxU6MBpI0PMU+ItI6MjgMjAqysJBB3BFYPBOKI7vZRy4Qkox1BXSAH5gAjkJknatoGjHJSjsM46yoDA4RbVtLaTkQZVBJYhRssnUgDQdgKtI9EEgEsYgVyHHOLEI4RnEd9fMz5VzlmS4XJ0jhcuXwdmlwc6G6CdvSvLeF8ccMVe6yqQTmiSIExrvoDz9dq6ThXFmulktP8OsbOwG7Ge57cq5SzSTujosMWuzqMjc1PpTP5Go7RaBmJJ5wfyase9YAlTtJg89NqV6p+UDwfTKuammk2sEcxPl2oQpr2RkpK0eaSadMMCnmntjqKPOOlTYUAqk7A+lEUPMGpFxHekcRRb+hpFV7VF7kVO2JqFrvalSkGqI2tjpSFmiNymz07MNUIWBS/soOlGr0XvhU5SFRRE+EI5mq1y2RzNXjf0iq7pzqUn5JxXgp+Lq31pVKWHSnrexjUJFqdAOlVgTRA1ijaZbUgTpvTiqwajDmihstB/OlI61WzGizUalZYkUSle9Vc1LMadS2LTxyoCRUGalmq1DYm95TEios1LNTRWGKHE3URS7sFVRJJ0Apw9YvEIv4hbDA5LaC88FhLOXS2sgyPhZvlQ7JNGrw68zl2dCiSAgaQ7CAS7Kfh1MBd/DJ3gZvty6jBXdB+gCdpNxAPvWsp0/msD25uxg7gmMzWx1/WrEeinXkJPKszXxf8GoP5L+Tc4daTKjhELG2vjgZspAMT0kzHeq/EeLhGAAA01PmaLhxizb5RbTSZiEGk155j+Js/ikgmZ7nTWuOa9Ul5O2FJybfg1OO+0LsciNCkTO5npHrp3qtwxLb6srgnckypnfn56Cuc9+C3i2P06VcZ3gZ850GWSdNNANYH+xrz68Ueiw+NcO920KwZSMy9YnY960RiFwqo+HfO7qM5KiFGkqBy8U/ICsS7cLfESTtrvH4akwYZ2VBqWbKB3aI+49KX0So7zhXHDeXQeLUEDTWtdMUdyBoOROk89hWMnBRYZVEDQajmw6+fXkAa0cMc0AmZ766E/nKuTSNWSYXFZ4JgEjOBBBysdJEeXqelWZNVLKMtw6RAg9NIiPzrVsmvf6f8DxZ62H94aYuaamJrtRxsKe9NmoaakLDzUJempoqorHD0XvRURFNVRWyQvTZqClVRWSZ6AtQzSzGihsTAU9AxNNTTK0TBKIJUOLxoQGFZiN4EAdMzH9pqhbxt64JULbGpnckDsZI67Vz3N6GuEpwprPTE3UbKwW4IkkQpH7GrdrHKTDApzGfKs+WtG5aEwWiy0TEAZiQBvPKOs01u4rfCwPkQatx0Flp8pqQCs7jeOexbNxEV1QguCxBykwSBGp1HOhzolC3RdyUstSJqAeon1ostO4aEGWllqwEp8tW4alfLQLZAJYAZmiTGpyzEnnEn1NWmIAkkRSDCj3Ea0ZARGp0Arz3254kLpW2mqo2YkTqcgynQTBz7idCOtdP7VcQKDIoaAud2XUgBhAgawYbUbECuH4LhBfxCF2KhGW45ZhDKrB3RVBUjQgFpgAaxArzZcrba8I9GLGkr8noqIf7NEFSLMQYkEJEacwRXmeFvhXzlUcAE5XUMp02I/jWvUcCFLsqsWRkBBzKw/u6MN9I5V5NZtlbhQ/oLg/0Ag/UR86FkeSKYxjpJoz7prQw2L8GRoyltzMqQYny7VmtTo+hHTWo0W2bWpuH4v3eIskqWBceEGCSGGUTykmKybjkag7VEb5DK2YBlIKnuCCPrFTRWeu8fxjLcyNCOV1Visr4vCwYHxKZ6/p5ERTJxCQSXkAc1dRJJIEsYzaHaeVc7xj2nsYq8l1LbpeyIrEsmQZGJ8IIMk52EnlGlXsLdd2CSzBwF1uZlI6QbcEdYImO1c3E0nxbOtwl1XRXXUMBryaBEjtpHyor7hFLEExyUSxJ2AHWaojF/2dGa46ZJMDRYYAE5RoIPPvrqZmthvbTBuQM7LJiSj5R5tFeyElFanknHZ2T3nxLZcltEUmGLuHdR/eCLCmOmY79oLJbvSVRiQJBuXSPi55LaATHUwPOrGK9pMGiF2xNogRojq7mdoRJY7HlRcK47hsTHurgLH9LAo/o2/wAprWyMakNjhzyTdvO5mVCZrSqOhCN4vM1fW3+ak+p3qzdyopZiFVRJLQAB1J5CnthWAZSGBEggggjqCNxTui1KpSlkq2bYpsgp2DUqFKYpVvL2p47VbjqUslLJVwr2pslO5alIpTZKuOsCSYFRqVJ0NW4alYpSq4UpVbFqR47AC4ACYEyREzG3OOfTpVQ8GVQDbho3UhfF1hh8J71jPxy6EgjlAMTt+o96bC8YuBIJnodte8b/ADrx+9E9XtSNGziHzKqqWhYIjxg5nKjUwvhjy++hgMI2XxganWQAR6aEb66GuXwXFHtlmGpfct1mZq5b9obwMsqld9oj61LLEnjl4D4jhbgMMxKzAXULOWTlWZKjTXvRLZ90AzFgTsAYGvQg6xtVV+OXHeScq8l6Rzmmx/ErjmBlyDbTqNTPeY+Qoc4mlCRp2rzx4XYKfJyPnTcaxObCuj/ExRB/5Z3VZiOU7dqo8PxxRYH1n9qyvafjjWEtFbYebqEySPgObKAOZymDyiaozt0gca5Z3lzFoqF5GRVJJ2gATJnYQKbCY5HRHXZ1VxoQYYSsg7GOVeN8b9qsTiUZCVS2wClF8RgGfE51PygaetLh3tDiLAOS4wzQxkq0kDKNxppAgdNtK3UjHB7o2J6L6n+JrLHE3vMVtrlRTBc65u6jeOXfy34ngntHi7+TOyC2+dc5VR8OhEhgQ3y7+fW4C9at5LbXUBuSUUfqCiTHXn+CsvazcYqrL4tgbks3zj0JNRO7AgKZE+IEiQDO1U8XxRFJhhppGhmec1yeM9pDYS2LltpysjPmSHcAa7zDNJJIETsamr6Np12dNxHCLiMyMkOohHkHQwY6kTEgjpXFWcalq4oDeLOVuCNgdHJOaGWQdDzIOkCq1zi968yXEX3YUyzK8kKdD4pECM0SpMzrWdiXAdlE5Y3OpnUyepJ596xqn2Ll9HrOIx64RLd2M6FrdpoPwLcZpuaaEAlAAI006RwvELBXE4pQJJa4B/W+YH0+9S8JBKvh7lxDZ0LW38LDKQwi5oFPgBiSNNV1Nb2I4VYe611y6e8QZRmQJplAcMRroI17/LKyRiqsnjk22ZvDfYwBFOJcZ4/7aNqOgdhzjpEdTQcS9m8MEzozoBuQc6D/ABK3i+ulRf8Ax+IFzLZue8Ibwq+VHUGdGloZYiGDHnoK0z7JY0+O9etoSplXYkDnrlEa/OpvzZKvKOSbgbMl0o6ubcNCz40jMXX5HbsRvve4f7MI2CxNy4v/AFFMp1VbeRmMR+qSJ6EfNsfw5sNdQPlKH4XViyMpPiAcR8M7EDlpzrpMRbREQA50YZTBmEbKGIPKc0xqPCDGkV1i7VnOXBwODXLdUiu1wTojF7jBERWZmZQSAFJgwATJ0gbzWA/CmzqyRl0aDKkA7gzpuTzq7xlM+HdF+LKAF1nMCrZQDzgrp/5UPtMU7RzPGONNiGe7JUBgLamDlQDYDYTufPfQU7s6YdHCl85fO4XMQwMBJ/QIM8iZ7VQTgt85fBGYwoJEkjeAN4gz0AJMAV0vs/7OXGtn3eIYZ2KqVVvdl1YyqudHkbldARuRNdJSjFWzlGLk6Oe4faV7im/my7BF0Y8oPNR9TXYcNthHYi3lVP8AtkNlcNG2rayNK5vj/ChYYtbu58rhSoIZgv6XdpkFmE7R4gJ2m/wa+7IWfUasCTEROo9PpWfyVp8GktXTXJ22A9qUePd3WVoEqxn/AOrSR51a9m8c1sXM8BGunJEALneDtoNWntBFcL7P8Hvq/v1ZCiIXK3C2UKwJjINwNTuvrXX8NxLMzs6AsXDADZc4BUKdCT/IrjLMoSq/8OscLlFs7BMYjTlYEjl/zVDifEPd5bgVnAzKUHxEEKS4HVYgzyY7bGmeKZDssyAROs9gtUsZdS8yZlY5HzwvhRZUqRLCSCJ6Vzy+uxpOnybx+jm2rXB0CYgqCSJZvFG6r4QAusEjSdtyajwHFPeKzBIysyNDAjMjFTB0kSKzrl975yIAninNlUvAMyu6jzM+VWcPktN4YGfXJAXK40LBRGjGSSOfnWoesjJpLn7ZmXpnFO+/o1kuA7U90EiAYqp788tKSXiDPLpXp92Jw9ti/szDYyPIfuaZLLR4WifPb10p72MHL/fvVK5xFRsh07xpR7kR0ZqWgQIYg9/5k0qx140p0e20cjqaen3oh7UjMbCiNJ9ae1hPPpy/irGUwOv8fm9WrVvQdSe/oK+Rcme/gp/2cAgEHbqKdMIv9067eIfxV8W/FRlRHlWk2DopHCId0nTr/tUiYVOSfU1bVYFHWtpGSFbCz8A+tcv/AP0TBO+GVkE+6cOQoJJUgqTHafSa6HF32R0kqqEGTrmLyMg7CA0mKks3nIcsyZC3gyg5imUHxg6qwMjX0rUZuD2KUdlR5R7O+zFzFIzwuaQUBIXPr4t9NoAGm3rZveziMmQr7q6vhMg7r/eXv1H/AD6WvC0XREVANRkyqNROaFgASDrWL7SYbKfeMoD5kzMd2WcokjfeZHSu0fUuUjCwpLk5LBG1asArdA92y++V0JdmLCVRZgQCQJrYfjODS57xA91ymRFcKVGfQtmOoYaiOjeVY+M4U183ktZc7XA+pgHX4QY1b4frWXdwtzDOFaWC6SNlIAJGv6paY6EV6XTMxteDT4vYW0PfW3co+mViSyPq0dwQDB7VyXE8Qz5WYsSJHiM94A5DeuytYm3ifBcKoiwzZ2KAmYGqgxvO1c3xbhgS61sGIgqJnQ7+LtrrRGXNMzlX10Tu4W1lURoPEJkqwEgnbcAjyqqmd3UASzH5SI37CNewrS4ej3GFpAhZ/iBkqqCJLDkB+cql4xwY2LiBHDBwjiRrLOVgjmJU+tDmk68jq2rOgx2ED29F8I2IHxGNWPn07a71h2veqAjNmS2r5VJOVldpiP6j5Qa3+J271i2AbishZUQBcpD5vDLFiADBFcdxTiRc76ZQNO2vy1NeOCk5Uuj0SlFK32afsPxDLiUQ/rlJJMa+JfIgj6mvQcTxNnwzyYZARHPXQ7+deN8IxITEW3Y6K6sT2nX6V6FjXYh0tNDlyGbQxrmETpqpQz3rr6iLTRyxO0Q2Qt201hzBYoytvldDqYkaFSy7860DhGVcizcTIF8AOdlI/wCokMdGMlVAMA3cxjKc2BhrufxHQg5WjTUzJHSRW7wTDuQ8Ivu2tsC7CYcoVygEmZG+hGo8qotw4/sZRU+SyLKoiteddchYg6NPifKJ1VjJUjk2sRrk28ajFVZ1OYsHynLmLICVB3VWum7J3y3jzVTUvHmZ7ZVzD27gLAFspDpoQJgEhwxO/UneuWs4MuCVBgH4gCY7eetayZGnw+CxYYtfJcnW27Chj77xL7vPcVfCt+7JK2iQZXDJAC20+KJI6Q28WoYXPeO99yQWMBEtwSiWU1CouYfNR4Rz5zLlMMzZgJgmGy9QCZq/whFNwKf7pcgh28C6sfAysdj8JnQkA6A8nLJP433+jsseLH8q6/Zr+12MezhlDHEnM6oq3FS3YKDV19ySXbw6ZyomZFefcMuHMQrGCreGd51APzium9tLbNetQBkKNkyINHLS4abaMf06sCdfiOsYvBLiI/8A1pKFV8WpABgER56ajrXt/FUvB4F8nbOt4ZxH3FhnyszPCDNAt5kzEeKZjU69RXS8KuKhZnCksULLHhQxLsCfMRXHYnjNwKlvJbCKQUZcxnpE9uRnlERVjAe0xR1VwWSII+WXn5V5skN+V2eiM9eGekcZ4Qgtk2wFMhhyBM7TyqkOHZbbMo8REANp4mOUKOszXKv7Tk3shJe0QIDAK6swiVdIIrZ4XccMjOSVnISTOpuBh32A+leLN6aKldHoxZpa1ZrYPCm2q6DMRJ2JAI+EmY6/SpntgwSqkqZUlRKkiJBjTTpUhxBcA5dwdgdVByrPIHQ69IqC4phQSQweTEEZRpkJ133kdu9dccdI8dHGUnJ2+wyWoHY0sx/O29C1zWl5GWpWusfv1qqykmP5rScyOnp6Ugmv5rQ5sUjMTCmeflT1p+779vSno3kPBVA/O1Sl+cctPz85VAbo1Ag6aa6zpuKUlhI00mO4P81ixotKYmTziiLSJ6eX51qs0k/nX671Ibhj5a9Zj+Z9DSmDRZUjrA1+1Jjsfzciq6SQO8yeQ/jamZxIXXn5z3pYD43CpdQo85TlMgwQQcwg+YrlsUzWsS62lcwqOQMzRPM9swO/XsK662pjsP2qquEWXddGYQT1CzANT5VCnTMHEcXLMh+BSCYOxEjQQZ7ctvU+K4wXcMzIMxABQawRyA0nWDQcQ4AzpeAOVs+dGGwUoudZ/wDIg6eRo2wxXKRs4BdTssSfBMaAsfl8pIx1akjbakqOJwvtK6h1NsKzySwBDBiSRHYeGOehovaPjyPh0UhjdOrMwOYbeIEjWRmE9zXaX8KcwygwDr0AP4a5f2k9nnuXC6DNoBM7ECIHafvXthmjJq+DzyhJJ07K3DcKlvCI7kzcuBlAiQFBEnnHib1pe0HCCEtvDAuh8Uyqljs5gETI1I5xSXheIC2CiM/ug5YAE+IxOh6iI8jXQYZ7mJcK6BUCmAVAGYiJA6wx170SnTtfsFG1TOb9h7WS5m2LB0+aFGI+tdb7Q4VHOHdSAUewDEFSPetIPTV6vYf2WRfdlIGQ5oiZzKBz+Wvb51bxnCg4QBoCuHaBvlJIGm3ijXtXGc9pbI6xSS1M7iytdCYdYm64MkwFRCHZifQf1Vydj2MDS2YhdQAZnNm1kgHlpXfW8LkZ3OrFdNNlEkAa7a69afD2FVYaZBY6bnnOkdTWY5JRVRKUYyds4O57JrbS4zKWOZWQ9BI0IB13NWcAr/2iVVsjqrDQx4UXn1kEV2DYQODM6iPz5A61Ph+HKSFHIaDUabzT7kmntzwWsVVcUcFhcOyXXRVYePwypkg5oIHOcun3rpcMjLnhzkKqCBEZgS2vQwpgjrrtWticMAUYSWVwNBPh1B+cT61OmBAUwNAScsAa5QIB+VTntz+iiq4OHxqf2jE5bjlUNxFbKurNlyKJO2lv5TPSNqxh1tqlpF+ESY2mevPc69q1RgQzvsAygGBzWSpHSp1w6ga6x28te+wqc/jqh/6s4njPCHe+HQaG24PyRo+uWqNqy6NYvpqSiKAJBDEwBI2kNXo9vDAgkgEwY/PrWZh+EoVtKXI90wYAAakE5ZPTX79qYzaq/ANJpnN8WttkUDUpc03JKhToOjAnQ9oPSp8Twz3lh1ygF1UtAAOjKx2/wyJ7VvHAgA5paSWE8iZb8NWLNsCeg+sn67/WqeRtprwEYJJ/s4vifCSMKGCyyPmI6y0EfUH5VbwvALbKC+eRI8DASwUGfEp0O0eVdHctIQUIlSZjcHWRt3APyq7hcOpXN3Ij5KdutZeWTVIVFds4BuFsjXMstlZHRgC0AFpQ6dm/9ZNatzEYtTaKojIGV2QuwdiRAlwBlGg2nbmK69MMsnwyNojQ7j670OMwSMjIZ8S5SVOVoMjwnkRFPuO7oNV0ZXBfa9L7iwUNhlLKMxDJ/wBOQVRlEMQBtp9Nd67cDTG3LuN6zLXDEW4jhQq20KKoAAXMwkjpooHrWg6DYfmn561rJl3VeDMcdBcp05j9qrI+p5efnEVZuHloAR9df9qr5OYMzM/T6a/WuD7OiAzz5b+nXpU7tse0fn1qBm6b9Oup19fvUl1oB2n9iNfn/IoYhvejQU1Vrbxr576dNNegpUWyHyjznfv+H5UIBjQCNB6c4H2o4PWJ+momJ5UXuRAOmninuByFIjAajQyCPLcydtRpT94nXSfp9z9anCaRzMeusn6UGIMZVG55jpqaegskTYx01jnv+01DYBJk+Y6xB07cvw1JaQhesH8594pnIn4htqNd9OXMiqyJc4XyJMz2/wCKktjaeZPz0OnrHrVOCW1BIBJ0OnMCemxmrCbyTrMCPU/tQm7BhDD/AN7l8wdZn1gVFiMMDAImZ18tNepqRp13/wCPwVFdflBOgj56xy3H3qZIrvZAGnMgTzyyB9j9adUE99Inudv3+VACW7EETO06/wAD0q0LHh13j55YgbU1ZNkKWhMHQEzznoQfWmtoIgjYkkwP6gB6VObZOvYxr3gfnnUfuyYK89eX0/OdQktxwoA0BJMfnpQh/FAO8DfvufrUj2TBO/b10/O1Q27RBEmd9uw5/Olgh2j4uZ69P+KicE9TG8/c9pqwyjYaAnXb8j4vShNvU6gaRz8x6a+tQkNrfrv15fSjz7xInfqNNhTYcDOQDz+XXl1mjuW4PPefWBHr9qCIkdtBO+5+fSrNtvDtvsPLXblrUYtEazsdNupmD5fepLNuCROpJ7ffferoGDaXLv8AFHp1+9QXd+pJIjrzH0qZ74LEAagRvv8AP1oVBnNOmveJPX5gdqhDsiIXznty/f6UsoHy08hT2nECAYmfLTT70n0BJO1Xgz5ILqfD0MyIO/8AvNVbnIA/qnlE/Lz+lWr7HUDbTlJmIG0az5VWKcm5Hl2BE/Wg0RsOu2hkDqZkRz2FT4dSNAeZJ6E/PapCQBpymexOsQKVrVdN539REc+XrVREtm4dJ57TOnLfv+1S3kzHoOvlAH71Dl5cgeXU+XOZ9ale9lWSJB2E/nKkh3iNNBp85E/uKjtNJknQaT32+lNnkCNOnyB0jnqaJWAIU7jaBHPf5zQRIQCoJ039CZ/emgHTaPpRZpG0a/T1qO62jdYHqT/sfSlgisdAJ3nf9tfOaDU5iNdZ6mYUfv6Ur2Yyd+RB2178zAFTWVUGT1zSdiSZA7TP0rIlcoDodYiRtyga/X81VWHYSATH4aVVFZFZ3NWh8Z8v5pUqY9ix1+N/8P8AFLEfCfIfempVS6YDYX4V86q2/wDuN+c6VKs+BLv6T+czSw+w8/8ASKalWgLF34G8j+9V8R+r5fc01Kh9gh76jxadfsKlu7N2Gnbw0qVKEa38B8v3NCn/AOZ+1PSpQPsl/S35yoTuPl/mpUqWSCf9Pl+5p13+Y/alSpAp4bdv8X80d3l5f6zTUqwhYunkf81SEaL/AIR96VKpCRR4vX7mhu/B/Sx+fWlSoXkhsHunn+61YxW3y/1GlSra/EH2Vjz/AKqjs7v5j7ilSrK6Qoe7/q/int/z/lNNSpZeA7W/9P7Ud1RlGn6v9LUqVC6ZALuvmf2oP1r5D7mlSqfReS9hfgHz/wAoqC58R/p/elSrT6QIr4n4f/X71ZYbf0/YUqVHkfBFb/c/alSpVkT/2Q==")